In [1]:
import torch
from torch.utils.data import DataLoader, random_split
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
from datamodules.mat_datamodules import MatDataModule
from models.lit_unet import LitUNet
import torch.nn.functional as F
from torchvision.transforms import Compose
from loss_functions.WeightedMSELoss import WeightedMSELoss
import torchvision.transforms as transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2

In [2]:
# class ZeroToMinus9999:
#     def __call__(self, tensor):
#         return torch.where(tensor == 0, torch.tensor(-9999.0, dtype=tensor.dtype), tensor)

In [3]:
# class MaxAbsScaler:
#     def __init__(self, max_abs_value):
#         self.max_abs_value = max_abs_value
#     def __call__(self, tensor):
#         return tensor / self.max_abs_value

In [4]:
# max_abs_value_input = 1.2040270566940308
# max_abs_value_tgt = 0.7881543636322021

# max_abs_scaler_input = MaxAbsScaler(max_abs_value=max_abs_value_input)
# max_abs_scaler_tgt = MaxAbsScaler(max_abs_value=max_abs_value_tgt)

In [5]:

train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=15, p=0.5),  # 最大15度の範囲でランダムに回転
    A.Affine(translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, p=0.5),
])

In [6]:
val_test_transform = A.Compose([
    
])

In [7]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_loss',  
    dirpath='checkpoints/UnetPlusPlus/flip/',  
    filename='best-checkpoint-{epoch:02d}-{val_loss:.4f}',  
    save_top_k=1,  # ベストモデルのみ保存
    mode='min',  
    save_weights_only=True
)

early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=5,  
    mode='min'  
)

trainer = Trainer(
    max_epochs=10,
    callbacks=[checkpoint_callback, early_stopping_callback] 
)


Trainer will use only 1 of 2 GPUs because it is running inside an interactive / notebook environment. You may try to set `Trainer(devices=2)` but please note that multi-GPU inside interactive / notebook environments is considered experimental and unstable. Your mileage may vary.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/home/ito/pytorch/nk/.nk/lib/python3.11/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:76: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default


In [8]:
datamodule = MatDataModule(
        data_dir='data',
        train_transform=train_transform,
        val_test_transform=val_test_transform,
        batch_size=32,
        num_workers=4,
        seed=42
    )

model = LitUNet(learning_rate=1e-4, loss_fn=F.mse_loss)

trainer.fit(model, datamodule=datamodule)

trainer.test(model, datamodule=datamodule)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name  | Type         | Params | Mode 
-----------------------------------------------
0 | model | UNetPlusPlus | 42.0 M | train
-----------------------------------------------
42.0 M    Trainable params
0         Non-trainable params
42.0 M    Total params
167.924   Total estimated model params size (MB)
73        Modules in train mode
0         Modules in eval mode


Epoch 1:   7%|▋         | 21/282 [00:17<03:41,  1.18it/s, v_num=67, train_loss=0.00204, val_loss=0.00168] 


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

: 

In [ ]:
from datamudules.mat_datamodules import MatDataModule

In [ ]:
a = MatDataModule(
    
)